In [ ]:

import numpy as np
from concurrent.futures import ProcessPoolExecutor
from scipy.stats import norm

def compute_var_montecarlo(sim_fn, S0, k, sigma0, n, m, T, alpha, rho, confidence=0.95):
    """
    General Monte Carlo VaR computation using any simulation function that returns individual path results.
    
    Returns the Value at Risk (VaR) for the given confidence level.
    """
    # Simulate prices or payoffs (we'll define the modified version of sim_fn next)
    payoffs = sim_fn(S0, k, sigma0, n, m, T, alpha, rho, return_all=True)
    
    # Compute P&L = Final value - Initial value (you can also use returns)
    returns = (np.array(payoffs) - S0) / S0
    
    # Compute empirical quantile (VaR)
    var = -np.percentile(returns, (1 - confidence) * 100) * S0
    return var

def var_black_scholes_montecarlo(S0, sigma, T, alpha=0.05):
    # Simulation
    N = 100000
    WT = np.random.normal(0, np.sqrt(T), N)
    ST = S0 * np.exp((-0.5 * sigma**2) * T + sigma * WT)
    # VaR calculation at level alpha
    VaR_alpha = np.quantile(ST, alpha)
    VaR_loss = S0 - VaR_alpha
    return VaR_alpha

def var_black_scholes(S0, sigma, T, alpha=0.05):
    """
    Calcula el VaR paramétrico bajo Black-Scholes para un activo.

    Parámetros:
    - S0: precio actual del activo
    - sigma: volatilidad anual
    - T: horizonte temporal en años (por ejemplo, 1 día = 1/252)
    - alpha: nivel de significancia (por defecto 5% → 95% confianza)

    Retorna:
    - VaR en unidades monetarias (no porcentaje)
    """
    mu = -0.5 * sigma**2 * T
    z = norm.ppf(alpha)
    log_return_quantile = mu + z * sigma * np.sqrt(T)
    ST_threshold = S0 * np.exp(log_return_quantile)
    VaR = S0 - ST_threshold
    return VaR

# Black-Scholes call price
def BS(s0, k, r, sigma, T):
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return s0 * norm.cdf(d1) - k * np.exp(-r * T) * norm.cdf(d2)

# Simula una sola trayectoria de Monte Carlo estándar (con volatilidad estocástica)
def std_MC_single(i, S0, k, sigma0, m, T, alpha, rho, W1_path, W2_path):
    sigma = np.zeros(m+1)
    X = np.zeros(m+1)
    sigma[0] = sigma0

    for j in range(m):
        sigma[j+1] = sigma[j] + alpha * sigma[j] * W1_path[j] * np.sqrt(T/m)
        X[j+1] = X[j] - 0.5 * sigma[j]**2 * T/m + sigma[j] * (rho * W1_path[j] + np.sqrt(1 - rho**2) * W2_path[j]) * np.sqrt(T/m)

    S = S0 * np.exp(X)
    payoff = max(S[-1] - k, 0)
    return payoff

# Versión paralela
def std_MC_parallel(S0, k, sigma0, n, m, T, alpha, rho,return_all=False):
    W1 = np.random.normal(0, 1, (n, m))
    W2 = np.random.normal(0, 1, (n, m))
    
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(
            std_MC_single,
            range(n),
            [S0]*n, [k]*n, [sigma0]*n, [m]*n, [T]*n, [alpha]*n, [rho]*n,
            W1, W2
        ))
        
    return results if return_all else np.mean(results)

# Individual simulation (for one path)
def cond_MC_single(i, S0, k, sigma0, m, T, alpha, rho, W_path):
    time = np.zeros(m+1)
    sigma = np.zeros(m+1)
    sigma[0] = sigma0
    for j in range(m):
        time[j+1] = time[j] + T/m
        sigma[j+1] = sigma[j] * np.exp((-0.5 * alpha**2) * time[j+1] + alpha * W_path[j] * np.sqrt(T/m))
    
    int_du = np.sum(sigma**2) * T/m
    rt_var = np.sqrt(np.mean(sigma**2))
    int_dw = np.sum([sigma[j] * (W_path[j+1] - W_path[j]) for j in range(m)]) * np.sqrt(T/m)
    
    S0_prime = S0 * np.exp(-0.5 * rho**2 * int_du + rho * int_dw)
    sigdef = np.sqrt(1 - rho**2) * rt_var
    return BS(S0_prime, k, 0, sigdef, T)

# Paralelizado
def cond_MC_parallel(S0, k, sigma0, n, m, T, alpha, rho,return_all=False):
    W = np.random.normal(0, 1, (n, m+1))
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(
            cond_MC_single,
            range(n),  # path index
            [S0]*n, [k]*n, [sigma0]*n, [m]*n, [T]*n, [alpha]*n, [rho]*n,
            W
        ))
    return results if return_all else np.mean(results)


price = cond_MC_parallel(100, 80, 0.15, 100, 1000, 1, 0.4, 0.6)
print(price)

var_cond = compute_var_montecarlo(cond_MC_parallel, 100, 80, 0.15, 1000, 1000, 1, 0.4, 0.6)
print("VaR con MC condicional:", var_cond)

result = var_black_scholes(100,0.2,1)
print(result)